In [1]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE" # on NERSC filelocking is not allowed
import h5py
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import pickle as pkl

import tensorflow as tf
# Make notebook run on other GPUS. GPT's solution ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')  # change 1 to 0, 2, 3 as needed
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

import sys
# Path to dir model.py lives in -------
# NOTE: This needs to be modified to where your repo lives, path to /repo/path/VAE_FS/models/
# If the jupyter notebook kernel is running from VAE_FS/models/ the
# line below is not needed
sys.path.append('/global/homes/j/jananinf/projs/VAE_FS/models/')

# import the custom models and functions
from models import VAE_Model_ATLAS_beta, Qmake_encoder_set_weights, Qmake_decoder_set_weights
# in VAE_0 we are using the beta cyclical annealing from Kenny's repo

2025-07-11 08:21:22.511765: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-11 08:21:23.937268: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load data
home_path = "/global/cfs/cdirs/m2616/jananinf/projsIO/VAE_FS/" # Updated to NERSC
file_path = home_path + "preprocessed_SNL_data.h5"
with h5py.File(file_path, 'r') as hf:           # Shapes:
    X_train = hf['X_train'][:]                  # (3200000, 57)
    X_test  = hf['X_test'][:]                   # (800000,  57)
    Ato4l_data  = hf['Ato4l_data'][:]           # (55969,   57) Signal data 
    hToTauTau_data  = hf['hToTauTau_data'][:]   # (691283,  57) ""
    hChToTauNu_data  = hf['hChToTauNu_data'][:] # (760272,  57) ""
    leptoquark_data = hf['leptoquark_data'][:]  # (340544,  57) ""
    print("Data loaded from preprocessed_SNL_data.h5")

Data loaded from preprocessed_SNL_data.h5


In [3]:
INPUT_SZ = 57
H1_SZ = 32
H2_SZ = 16
LATENT_SZ = 3
NUM_EPOCHS = 100
BATCH_SIZE = 1024
STOP_PATIENCE = 15
LR_PATIENCE = 10

# enc = Qmake_encoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
# enc.summary()

In [4]:
# dec = Qmake_decoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
# dec.summary()

In [5]:
# steps_per_epoch = X_train.shape[0] // BATCH_SIZE
# vae = VAE_Model(enc, dec, steps_per_epoch=steps_per_epoch, cycle_length=10, min_beta=0.1, max_beta=0.8)
# opt = keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1000)
# vae.compile(optimizer=opt) # Not sure what weighted_mse is doing.



In [6]:
# # looks like early_stopping is needed for val_loss
# early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=LR_PATIENCE, verbose=1)

In [ ]:
train = False
NUM_TRAIN = 20 # For now lets just try 20
save = True
SAVE_PATH = home_path+f"/VAE_trainings/attempt3/atlas_beta/" # As of 7/10/25 should be synced with vae1_analysis
# Last save is in attempt 1. New save should go to attempt 2
# Attempt History. The original code for each folder should also be tied to the commits. 
# 0: no weighted MSE, no call_backs
# 1: adding ReduceLRonPlatueau and early_stopping and the test_step
# 2: /atlas_beta/. now training multiple times. But other wise no different from previous. This is to differentiate between the old cyclical beta and the atlas beta schedule.
# Changed batch size to match paper's 1024
# 3: Clipnorm changed. 

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=LR_PATIENCE, verbose=1)

for i in range(NUM_TRAIN):
    if train:
        tf.keras.backend.clear_session()

        print(f"TRAINING ITERATION {i} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
        enc = Qmake_encoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
        dec = Qmake_decoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)

        steps_per_epoch = X_train.shape[0] // BATCH_SIZE
        vae = VAE_Model_ATLAS_beta(enc, dec, steps_per_epoch=steps_per_epoch, cycle_length=10, min_beta=0.1, max_beta=0.8)
        opt = keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1000)
        vae.compile(optimizer=opt)


        # history = vae.fit(
        #             train_ds,
        #             validation_data=val_ds,
        #             epochs=NUM_EPOCHS,
        #             callbacks=[early_stopping, reduce_lr],
        #             verbose=2
        #         )
        history = vae.fit(x=X_train, validation_split=0.2, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping,reduce_lr], shuffle=True)
        # Iterative training. 
        save_path = SAVE_PATH+f"n_{i}/" # As of 7/8/25. Should be synced with vae0_analysis
        if save:
            print(f"SAVING ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
            vae.save_weights(filepath=save_path, save_format='tf')

            # Now save the histories
            with open(save_path + f"training_history.pkl", 'wb') as f:
                pkl.dump(history.history, f)




TRAINING ITERATION 0 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 1/100
2500/2500 [==============================] - 14s 5ms/step - loss: 1.8206 - reconstruction_loss: 0.9840 - kl_loss: 0.0952 - beta: 0.1560 - val_loss: 1.1118 - val_reconstruction_loss: 1.0277 - val_kl_loss: 0.0842 - val_beta: 0.1560 - lr: 1.0000e-04
Epoch 2/100
2500/2500 [==============================] - 12s 5ms/step - loss: 0.4939 - reconstruction_loss: 0.3983 - kl_loss: 0.0569 - beta: 0.2120 - val_loss: 0.9851 - val_reconstruction_loss: 0.9278 - val_kl_loss: 0.0573 - val_beta: 0.2120 - lr: 1.0000e-04
Epoch 3/100
 926/2500 [==========>...................] - ETA: 6s - loss: 0.4089 - reconstruction_loss: 0.3597 - kl_loss: 0.0405 - beta: 0.2327

Plot Loss vs epoch history.

In [5]:
import matplotlib.pyplot as plt
import pickle as pkl
# Assuming 'history' is the object returned by your model.fit() call

for i in range(2):
    save_path = SAVE_PATH + f"n_{i}/"
    with open(save_path + 'training_history.pkl', 'rb') as f:
        history = pkl.load(f)

    # Extract the loss values
    total_loss = history['loss']
    reco_loss = history['reconstruction_loss']
    kl_loss = history['kl_loss']
    val_total_loss = history['val_loss']
    val_reco_loss = history['val_reconstruction_loss']
    val_kl_loss = history['val_kl_loss']

    # Create a new figure
    plt.figure(figsize=(12, 8))

    # Plot training losses
    plt.plot(total_loss, label='Total Loss', color='blue')
    plt.plot(reco_loss, label='Reconstruction Loss', color='green')
    plt.plot(kl_loss, label='KL Loss', color='red')
    plt.plot(history['beta'],label="beta")

    # Plot validation losses
    plt.plot(val_total_loss, label='Val Total Loss', color='blue', linestyle='--')
    plt.plot(val_reco_loss, label='Val Reconstruction Loss', color='green', linestyle='--')
    plt.plot(val_kl_loss, label='Val KL Loss', color='red', linestyle='--')

    # Customize the plot
    plt.title(f'Training and Validation Losses Run: {i}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.yscale("log")
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.show()

NameError: name 'SAVE_PATH' is not defined